In [1]:
%run imports_and_functions.ipynb

from dbfread import DBF

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
# income data files come in .dbf, must install this to read from python

# !pip install dbfread

### Creating Final Businesses Dataframe within City Radius

In [6]:
df = pd.read_csv('PittsburghBusinessesUpdated.csv', sep=';', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6044 entries, 0 to 6043
Data columns (total 13 columns):
Unnamed: 0       6044 non-null int64
alias            6044 non-null object
attributes       0 non-null float64
categories       6044 non-null object
coordinates      6044 non-null object
distance         6044 non-null float64
latitude         6028 non-null float64
longitude        6028 non-null float64
main_category    6044 non-null object
name             6044 non-null object
price            2458 non-null object
rating           6044 non-null float64
review_count     6044 non-null int64
dtypes: float64(5), int64(2), object(6)
memory usage: 613.9+ KB


In [7]:
def dropduplicate_yelp_entries(df) :
    dups = df[df[['latitude','longitude','alias']].duplicated(keep=False)].sort_values('latitude')
    mask1 = dups[['latitude','longitude','alias']].duplicated(keep='first')
    mask2 = dups[['latitude','longitude','alias']].duplicated(keep=False)
    mask = mask2 & ~mask1
#     print(dups)
    for index in dups[mask].index :
        dupset = dups[(dups['alias']==dups.loc[index,'alias']) & (dups['longitude']==dups.loc[index,'longitude'])].sort_values(
            'review_count', ascending=False)
        dropindex = dupset.iloc[1:,0].index
        df.drop(dropindex, axis=0, inplace=True)
        
def getPriceRange(string) :
    regex = re.compile("RestaurantsPriceRange2\\': \\'(\d)")
    try :
        match = regex.search(string)
        return int(match.groups(0)[0])
    except : return 0
    
def convertpricecolumn(pricesymbol) :
    try : num = len(pricesymbol)
    except : num = 0
    return num

def get_main_category(catstring) :
    string = ''
    try :
        for cat in main_categories :
            if cat in catstring : string = string + cat + ', '
    except : pass
    return string[0:(len(string)-2)]

In [8]:
print(df.shape)
dropduplicate_yelp_entries(df)
df.shape
# NOT more businesses in Yelp for Pittsburgh for 2019 versus 2014

(6044, 13)


(6044, 13)

In [10]:
df['price'] = df['price'].map(convertpricecolumn)
df['price'].value_counts()

0    3586
2    1253
1     932
3     224
4      49
Name: price, dtype: int64

In [8]:
# df['main_category'] = df['categories'].map(get_main_category)
df['main_category'].value_counts()

Restaurants                                                                                          1225
Shopping                                                                                              594
Food                                                                                                  528
Food, Restaurants                                                                                     527
Beauty & Spas                                                                                         424
Nightlife, Restaurants                                                                                334
Automotive                                                                                            323
Home Services                                                                                         300
Health & Medical                                                                                      266
Active Life                                   

In [11]:
# SET FOR EACH CITY: maxradius, testlat, testlong
maxradius = 7000
testlat = 40.445503
testlong = -79.995499 # Pittsburgh, PA

In [12]:
df['distance'] = calcDistances(testlat, testlong, df)

df[df['distance']<maxradius].shape

(5420, 13)

In [11]:
df['distance'].sort_values(ascending=False)

2282    3.593705e+06
4088    2.589954e+06
3387    1.635559e+06
630     3.621119e+05
3430    1.521640e+05
1065    4.942384e+04
6726    4.723671e+04
1859    4.308922e+04
4749    4.288466e+04
6335    3.506270e+04
5532    3.264197e+04
2739    3.181519e+04
3413    2.830556e+04
758     2.564070e+04
4737    2.498600e+04
623     2.496990e+04
1762    2.484036e+04
6263    2.482073e+04
6375    2.471968e+04
6380    2.470969e+04
6012    2.470528e+04
2983    2.469766e+04
4933    2.464014e+04
6123    2.457933e+04
2859    2.446130e+04
4742    2.440304e+04
3683    2.418917e+04
5245    2.417284e+04
1225    2.409412e+04
6613    2.403808e+04
6212    2.403630e+04
5892    2.401614e+04
6055    2.396733e+04
1262    2.395974e+04
5628    2.395636e+04
729     2.395636e+04
6104    2.393179e+04
6592    2.393153e+04
6065    2.393077e+04
6279    2.393061e+04
5518    2.382560e+04
5939    2.376487e+04
5982    2.360399e+04
947     2.317657e+04
3153    2.312572e+04
6452    2.312572e+04
6388    2.311739e+04
2799    2.297

In [12]:
df[df[['latitude','longitude']].duplicated(keep=False)].sort_values('latitude')

,address,attributes,categories,city,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,price,main_category,distance
2851,NaN,{'BusinessAcceptsCreditCards': 'False'},"Notaries, Local Services",Pittsburgh,40.320570,-80.087016,Notary In Your Neighborhood,NaN,15241.0,3,5.0,PA,0,Local Services,15901.087008
149,NaN,NaN,"Pet Groomers, Pet Services, Pets",Pittsburgh,40.320570,-80.087016,Grand Slam Grooming,NaN,15241.0,3,5.0,PA,0,Pets,15901.087008
423,NaN,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Lighting Fixtures & Equipment, ...",Pittsburgh,40.320570,-80.087016,Pittsburgh Socialights,NaN,15241.0,3,5.0,PA,0,Home Services,15901.087008
4364,5843 Brownsville Rd,NaN,"Pet Services, Pets, Pet Groomers",Pittsburgh,40.327371,-79.986368,Woody's Dog Wash & Boutique,NaN,15236.0,13,4.5,PA,0,Pets,13140.480933
3733,5843 Brownsville Rd,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Greek, Restaurants",Pittsburgh,40.327371,-79.986368,Sparta Gyros,NaN,15236.0,4,3.0,PA,0,Restaurants,13140.480933
218,"5843 Brownsville Rd, Ste 2",NaN,"Professional Services, Payroll Services, Accou...",Pittsburgh,40.327371,-79.986368,Adams & Associates,NaN,15236.0,3,2.5,PA,0,"Financial Services, Professional Services",13140.480933
3685,530 E Bruceton Rd,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Beer, Wine & Spirits, Seafood, Nightlife, Rest...",Pittsburgh,40.328073,-79.946763,Wintzell's Oyster House,NaN,15236.0,50,3.5,PA,2,"Food, Nightlife, Restaurants",13680.552496
3862,530 E Bruceton Rd,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...","Restaurants, Sandwiches, American (Traditional...",Pittsburgh,40.328073,-79.946763,Bruceton Grille,NaN,15236.0,16,2.0,PA,2,Restaurants,13680.552496
807,"10 Old Clairton Rd, Ste 2328f","{'AcceptsInsurance': 'True', 'BikeParking': 'F...","Doctors, Eyewear & Opticians, Optometrists, He...",Pittsburgh,40.339812,-79.965456,America's Best Contacts & Eyeglasses,NaN,15236.0,6,4.5,PA,2,"Health & Medical, Shopping",12010.107400
1733,10 Old Clairton Rd,"{'Alcohol': 'full_bar', 'BusinessAcceptsCredit...","American (New), Restaurants",Pittsburgh,40.339812,-79.965456,Brass Rail,NaN,15236.0,3,3.5,PA,2,Restaurants,12010.107400


In [13]:
rowstodrop = df[(df['distance']>maxradius) | df['latitude'].isnull() | df['longitude'].isnull()].index

df.drop(rowstodrop, axis=0, inplace=True)
df.shape

(5420, 13)

In [14]:
df['lat_index'] = round(df['latitude'], 3)
df['long_index'] = df['longitude']

df = df.set_index(['lat_index','long_index'], drop=False).sort_index()
df = df.sort_index(level = [0,1])

In [16]:
df.head()

address  \
lat_index long_index                           
40.384    -79.995502  2435 Saw Mill Run Blvd   
40.385    -80.015724         3049 Sussex Ave   
          -79.994480  2351 Saw Mill Run Blvd   
          -79.978469     2616 Brownsville Rd   
          -79.978421     2614 Brownsville Rd   

                                                             attributes  \
lat_index long_index                                                      
40.384    -79.995502             {'BusinessAcceptsCreditCards': 'True'}   
40.385    -80.015724  {'Alcohol': 'full_bar', 'Ambience': "{'romanti...   
          -79.994480            {'BusinessAcceptsCreditCards': 'False'}   
          -79.978469  {'Alcohol': 'none', 'Ambience': "{'romantic': ...   
          -79.978421  {'Alcohol': 'none', 'BusinessAcceptsCreditCard...   

                                                             categories  \
lat_index long_index                                                      
40.384    -79.995502        Automotive, Auto Customization, Car Dealers   
40.385    -80.015724  American (Traditional), Barbeque, American (Ne...   
          -79.994480  Automotive, Car Dealers, Used Car Dealers, Aut...   
          -79.978469             Sandwiches, Mediterranean, Restaurants   
          -79.978421                               Chinese, Restaurants   

                            city   latitude  longitude  \
lat_index long_index                                     
40.384    -79.995502  Pittsburgh  40.383611 -79.995502   
40.385    -80.015724  Pittsburgh  40.384600 -80.015724   
          -79.994480  Pittsburgh  40.385131 -79.994480   
          -79.978469  Pittsburgh  40.384954 -79.978469   
          -79.978421  Pittsburgh  40.385088 -79.978421   

                                             name neighborhood  postal_code  \
lat_index long_index                                                          
40.384    -79.995502          Leopardi Auto Sales    Overbrook      15234.0   
40.385    -80.015724             The Getaway Cafe          NaN      15226.0   
          -79.994480               Used Car World    Overbrook      15210.0   
          -79.978469           Sam's Sun Sandwich      Carrick      15227.0   
          -79.978421  Bai Ling Chinese Restaurant      Carrick      15227.0   

                      review_count  stars state  price  \
lat_index long_index                                     
40.384    -79.995502             3    2.5    PA      0   
40.385    -80.015724            48    3.0    PA      2   
          -79.994480             3    1.0    PA      0   
          -79.978469            34    4.5    PA      1   
          -79.978421             4    5.0    PA      2   

                               main_category     distance  lat_index  \
lat_index long_index                                                   
40.384    -79.995502              Automotive  6872.615887     40.384   
40.385    -80.015724  Nightlife, Restaurants  6977.320873     40.385   
          -79.994480              Automotive  6704.423174     40.385   
          -79.978469             Restaurants  6877.076578     40.385   
          -79.978421             Restaurants  6863.408863     40.385   

                      long_index  
lat_index long_index              
40.384    -79.995502  -79.995502  
40.385    -80.015724  -80.015724  
          -79.994480  -79.994480  
          -79.978469  -79.978469  
          -79.978421  -79.978421

In [15]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

def categories_to_alias2(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    rawlis = list(set(catstring.split(', ')))
    lis = []
    for item in rawlis :
        if 'alias' in item :
            alias = item.split(':')[1].strip().strip("'")
            lis += [alias]
#     print(lis)
    lis = list(map(get_alias2,lis))
    lis = [item for item in lis if item != '_other']
    lis += ['_' + dfrow['main_category']]
    # some new categories are not in category dictionary, and each row already has main category
    # so am taking out all instances where business category could not be found
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias2(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat_alias_dict[cat])])
    except : alias = '_other'
    return alias

def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat)])
    except : alias = '_other'
    return alias

## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [16]:
df['alias_cats'] = df.apply(categories_to_alias2, axis=1)

In [19]:

df.to_csv('PittsburghBusinessesFinalUpdated.csv',sep=';',header=True,index=True,compression='gzip')

In [18]:
df['alias_cats']

lat_index  long_index
40.384     -79.995816                             0_automotive 0_bodyshops
           -79.995502                           0_car_dealers 0_automotive
           -79.995050                           0_car_dealers 0_automotive
           -79.978040                                       0_homeservices
           -79.977830                                          2_beautysvc
           -79.977650                                0_health 0_physicians
           -79.977360                                1_pizza 1_restaurants
40.385     -80.015711                                   0_hair 0_beautysvc
           -80.015648                     0_plumbing 0_hvac 0_homeservices
           -80.000230                                       0_hotelstravel
           -79.995590                             0_automotive 0_bodyshops
           -79.995376                                       0_homeservices
           -79.994780                                   1_bars 1_nightlife
   

### Census Block Population and Income to X_train Coordinates DataFrame

In [24]:
table = DBF('tl_2014_42_tabblock10.dbf')
type(table)

dbfread.dbf.DBF

In [25]:
df_coords = pd.DataFrame(iter(table))
df_coords.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,42,109,070100,2030,421090701002030,Block 2030,G5040,U,85762,C,S,0,1698,+40.8345322,-076.8419872
1,42,081,010100,1271,420810101001271,Block 1271,G5040,R,,,S,5502435,3053,+41.4359798,-077.1537260
2,42,081,011000,3019,420810110003019,Block 3019,G5040,R,,,S,79,81303,+41.2612186,-076.9348230
3,42,081,010700,1006,420810107001006,Block 1006,G5040,R,,,S,1375345,13226,+41.3299120,-076.8104500
4,42,081,011700,2008,420810117002008,Block 2008,G5040,R,,,S,0,121919,+41.1644590,-076.9537720


In [26]:
df_coords = df_coords[['GEOID10','INTPTLAT10','INTPTLON10','UR10','ALAND10','AWATER10']]
df_coords.to_csv('PittsburghCoords.csv',sep=';',header=True,index=True,compression='gzip')

In [27]:
def cut(geoid) :
    index = str(geoid)
    return int(index[0:(len(index)-3)])

def matchincome(geoid) :
    try : return blockgroupincome.loc[geoid,'income']
    except : return 0

def matchpop(geoid) :
    try : return blockgrouppop.loc[geoid,'population']
    except : return 0

In [30]:
table = DBF('tabblock2010_42_pophu.dbf')

populations = pd.DataFrame(iter(table))
populations.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10
0,42,001,030500,2075,420010305002075,N,1,1
1,42,001,030400,1048,420010304001048,N,3,6
2,42,001,031300,1027,420010313001027,N,7,14
3,42,001,030400,1071,420010304001071,N,5,6
4,42,001,030400,1069,420010304001069,N,17,37


In [31]:
populations = populations[['BLOCKID10','POP10']]

In [32]:
populations.columns = ['GEO.id2','population']
populations['blockgroup'] = populations['GEO.id2'].map(cut)
populations['GEO.id2'] = populations['GEO.id2'].map(int)
populations.set_index(['GEO.id2'],inplace=True,drop=False)
populations.head()

,GEO.id2,population,blockgroup
GEO.id2,,,
420010305002075,420010305002075,1,420010305002
420010304001048,420010304001048,6,420010304001
420010313001027,420010313001027,14,420010313001
420010304001071,420010304001071,6,420010304001
420010304001069,420010304001069,37,420010304001


In [67]:
populations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421545 entries, 420010305002075 to 421330229104048
Data columns (total 5 columns):
GEO.id2          421545 non-null int64
population       421545 non-null int64
blockgroup       421545 non-null int64
income           421429 non-null object
blockgrouppop    421545 non-null object
dtypes: int64(3), object(2)
memory usage: 19.3+ MB


In [49]:
blockgroupincome = pd.read_csv('Pittsburgh.csv', usecols=[2,4,6])
blockgroupincome.head()

,GEO.id2,HD01_VD01,income
0,Id2,Estimate; Total,Estimate; Per capita income in the past 12 mon...
1,420030103001,2465,9985
2,420030103002,3429,5028
3,420030103003,337,12648
4,420030103004,315,25730


In [50]:
blockgroupincome.drop(0, axis=0, inplace=True)
blockgroupincome.columns = ['GEO.id2','population','income']
blockgroupincome['GEO.id2'] = blockgroupincome['GEO.id2'].map(int)
blockgroupincome.set_index('GEO.id2', inplace=True)
blockgroupincome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 420030103001 to 420039822001
Data columns (total 2 columns):
population    1100 non-null object
income        1096 non-null object
dtypes: object(2)
memory usage: 25.8+ KB


In [51]:
populations['income'] = populations['blockgroup'].map(matchincome)
populations['income'].value_counts()

0        391026
-           200
19939       143
41727       127
40252       123
30595       121
30165       120
32022       108
28067        97
39750        95
31081        95
45849        94
27369        90
10886        89
19735        89
48132        88
18910        86
15485        80
29516        80
19447        79
26104        78
20103        78
30311        77
42340        76
39209        76
16708        75
27484        74
57759        74
2867         73
42572        73
24808        73
38254        73
35442        72
31012        72
55141        72
67610        71
26593        71
32689        70
15704        70
24552        69
18315        69
15402        69
38625        69
38641        68
16213        67
21474        66
19786        66
41942        65
25894        65
16564        64
16656        63
15027        63
21630        63
27561        62
55764        62
22885        62
45321        61
51263        61
88378        60
34177        60
17246        60
19251        60
42777   

In [52]:
blockgrouppop = blockgroupincome

populations['blockgrouppop'] = populations['blockgroup'].map(matchpop)
populations['blockgrouppop'].value_counts()

0       391026
1149       208
943        205
1388       186
0          174
403        172
811        149
1073       146
1340       145
1564       133
620        131
3702       127
4077       121
826        121
1321       118
1141       118
740        116
1227       116
1699       113
1199       110
1179       109
1029       109
1315       107
825        106
1033       106
816        106
1638       104
1043       104
1374       103
832        103
1219       100
1807        99
905         98
1439        98
3409        95
1407        95
1370        94
269         94
1309        94
1154        94
1001        94
818         92
1228        92
890         91
2129        90
1106        90
622         90
1096        89
1639        89
786         88
1009        86
1191        86
1215        84
1742        83
717         82
361         80
1247        80
1688        80
823         79
1257        79
930         79
885         79
925         78
4           78
2606        77
511         76
1386      

### X_train Coordinates DataFrame

In [35]:
features = ['distance','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea','block']

In [36]:
def blocksdropna(df) :
    df.drop(df[df['income']==0].index, inplace=True)
    df.drop(df[df['income']=='-'].index, inplace=True)
    df.drop(df[df['population']<20].index, inplace=True)  # min 20 people for block to be counted

In [53]:
blocks = pd.read_csv('PittsburghCoords.csv',sep=';',compression='gzip')
blocks.head()

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,UR10,ALAND10,AWATER10
0,0,421090701002030,40.834532,-76.841987,U,0,1698
1,1,420810101001271,41.435980,-77.153726,R,5502435,3053
2,2,420810110003019,41.261219,-76.934823,R,79,81303
3,3,420810107001006,41.329912,-76.810450,R,1375345,13226
4,4,420810117002008,41.164459,-76.953772,R,0,121919


In [54]:
blocks.drop('Unnamed: 0', axis=1, inplace=True)
blocks.columns = ['GEO.id2','latitude','longitude','urban','landarea','waterarea']
blocks['blockgroup'] = blocks['GEO.id2'].map(cut)
blocks['GEO.id2'] = blocks['GEO.id2'].map(int)
blocks = blocks.set_index('GEO.id2')
blocks.sort_index(inplace=True)
blocks.head()

,latitude,longitude,urban,landarea,waterarea,blockgroup
GEO.id2,,,,,,
420010301011000,40.058694,-77.106766,R,66598,0,420010301011
420010301011001,40.059137,-77.110474,R,172195,0,420010301011
420010301011002,40.061111,-77.113522,R,39245,0,420010301011
420010301011003,40.062056,-77.118512,R,365660,0,420010301011
420010301011004,40.065134,-77.123494,R,59794,0,420010301011


In [60]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421545 entries, 420010301011000 to 421330240022037
Data columns (total 6 columns):
latitude      421545 non-null float64
longitude     421545 non-null float64
urban         421545 non-null object
landarea      421545 non-null int64
waterarea     421545 non-null int64
blockgroup    421545 non-null int64
dtypes: float64(2), int64(3), object(1)
memory usage: 22.5+ MB


In [69]:
blocks = blocks.join(populations[['income','blockgrouppop','population']], how='inner')
blocks.shape

(421545, 9)

In [70]:
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421545 entries, 420010301011000 to 421330240022037
Data columns (total 9 columns):
latitude         421545 non-null float64
longitude        421545 non-null float64
urban            421545 non-null object
landarea         421545 non-null int64
waterarea        421545 non-null int64
blockgroup       421545 non-null int64
income           421429 non-null object
blockgrouppop    421545 non-null object
population       421545 non-null int64
dtypes: float64(2), int64(4), object(3)
memory usage: 32.2+ MB


In [71]:
blocksdropna(blocks)
blocks.shape

(14550, 9)

In [72]:
blocks[(blocks['population']!=0) & (blocks['income']==0)] # must drop as don't have y target (income)

,latitude,longitude,urban,landarea,waterarea,blockgroup,income,blockgrouppop,population
GEO.id2,,,,,,,,,


In [73]:
testlat, testlong  # used as center of Pittsburgh

(40.445503, -79.995499)

In [74]:
blocks['distance_from_city'] = blocks.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],testlat, testlong)),         
    axis=1)

In [75]:
blocks = blocks[blocks['distance_from_city']<6000]
blocks.shape

(3070, 10)

In [76]:
blocks = blocks.reindex(['distance_from_city','urban','latitude','longitude','blockgroup','income',
                'blockgrouppop','population','landarea','waterarea'], axis=1)
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
GEO.id2,,,,,,,,,,
420030103001007,1224.507208,U,40.434602,-79.993324,420030103001,9985,2465,2594,41768,0
420030103002002,818.745429,U,40.438604,-79.992094,420030103002,5028,3429,69,9209,0
420030103002015,959.100895,U,40.437214,-79.992321,420030103002,5028,3429,605,23201,0
420030103002016,1156.609093,U,40.435811,-79.990506,420030103002,5028,3429,2662,83841,0
420030103003003,1144.493890,U,40.438364,-79.985769,420030103003,12648,337,32,5624,0


In [77]:
blocks[blocks['distance_from_city']<6000][['population']].sum()

population    216169
dtype: int64

In [78]:
def bootstrapBlocks(blocksdf) :
    df = blocksdf.copy()
    print(df.index)
    for index in blocksdf[blocksdf['population']>200].index :
        iterpop = int(np.ceil(blocksdf.loc[index,'population'] / 200))
        lis = [pd.Series(data=blocksdf.loc[index].values, name=index+(10**15*i),
                        index=blocksdf.columns) for i in range(1,iterpop)]
#         print(len(lis))
        df = df.append(lis) # more efficient and faster to append all at once
    return df

In [79]:
blocks = bootstrapBlocks(blocks)
blocks.shape

Int64Index([420030103001007, 420030103002002, 420030103002015, 420030103002016,
            420030103003003, 420030103003007, 420030103003008, 420030103003009,
            420030103003012, 420030103003015,
            ...
            420035632003064, 420035632003067, 420035632003069, 420035632003070,
            420039809001009, 420039822001000, 420039822001005, 420039822001007,
            420039822001012, 420039822001015],
           dtype='int64', name='GEO.id2', length=3070)


(3297, 10)

In [80]:
blocks.to_csv('PittsburghTrainRowCoords.csv',
          sep=';',
          header=True,
          index=True,
          compression='gzip')

## Vectorize Businesses and Assemble X_train

### Construct Category Corpus with Aliases

In [20]:
lat1000m = -115.161605-(-115.172713)

def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias_cats']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias_cats'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        try :
            distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
        except : # catch very rare attribute error to fill in rest of rows
            distances[index] = 10000 # if calcDist gives error, don't include this business
    distances = distances[distances<1000]
    return distances

def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
            coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                            coorddf.loc[index,'latitude'], 
                                            coorddf.loc[index,'longitude'])

In [82]:
df = pd.read_csv('PittsburghBusinessesFinal.csv',sep=';',compression='gzip',
                 index_col=['lat_index.1','long_index.1'])
df.head()

lat_index  long_index                 address  \
lat_index.1 long_index.1                                                  
40.384      -79.995502       40.384  -79.995502  2435 Saw Mill Run Blvd   
40.385      -80.015724       40.385  -80.015724         3049 Sussex Ave   
            -79.994480       40.385  -79.994480  2351 Saw Mill Run Blvd   
            -79.978469       40.385  -79.978469     2616 Brownsville Rd   
            -79.978421       40.385  -79.978421     2614 Brownsville Rd   

                                                                 attributes  \
lat_index.1 long_index.1                                                      
40.384      -79.995502               {'BusinessAcceptsCreditCards': 'True'}   
40.385      -80.015724    {'Alcohol': 'full_bar', 'Ambience': "{'romanti...   
            -79.994480              {'BusinessAcceptsCreditCards': 'False'}   
            -79.978469    {'Alcohol': 'none', 'Ambience': "{'romantic': ...   
            -79.978421    {'Alcohol': 'none', 'BusinessAcceptsCreditCard...   

                                                                 categories  \
lat_index.1 long_index.1                                                      
40.384      -79.995502          Automotive, Auto Customization, Car Dealers   
40.385      -80.015724    American (Traditional), Barbeque, American (Ne...   
            -79.994480    Automotive, Car Dealers, Used Car Dealers, Aut...   
            -79.978469               Sandwiches, Mediterranean, Restaurants   
            -79.978421                                 Chinese, Restaurants   

                                city   latitude  longitude  \
lat_index.1 long_index.1                                     
40.384      -79.995502    Pittsburgh  40.383611 -79.995502   
40.385      -80.015724    Pittsburgh  40.384600 -80.015724   
            -79.994480    Pittsburgh  40.385131 -79.994480   
            -79.978469    Pittsburgh  40.384954 -79.978469   
            -79.978421    Pittsburgh  40.385088 -79.978421   

                                                 name neighborhood  \
lat_index.1 long_index.1                                             
40.384      -79.995502            Leopardi Auto Sales    Overbrook   
40.385      -80.015724               The Getaway Cafe          NaN   
            -79.994480                 Used Car World    Overbrook   
            -79.978469             Sam's Sun Sandwich      Carrick   
            -79.978421    Bai Ling Chinese Restaurant      Carrick   

                          postal_code  review_count  stars state  price  \
lat_index.1 long_index.1                                                  
40.384      -79.995502        15234.0             3    2.5    PA      0   
40.385      -80.015724        15226.0            48    3.0    PA      2   
            -79.994480        15210.0             3    1.0    PA      0   
            -79.978469        15227.0            34    4.5    PA      1   
            -79.978421        15227.0             4    5.0    PA      2   

                                   main_category     distance  \
lat_index.1 long_index.1                                        
40.384      -79.995502                Automotive  6872.615887   
40.385      -80.015724    Nightlife, Restaurants  6977.320873   
            -79.994480                Automotive  6704.423174   
            -79.978469               Restaurants  6877.076578   
            -79.978421               Restaurants  6863.408863   

                                                                      alias  
lat_index.1 long_index.1                                                     
40.384      -79.995502                                 0_auto 0_car_dealers  
40.385      -80.015724    2_nightlife 2_bars 2_tradamerican 2_bbq 2_rest...  
            -79.994480                                 0_auto 0_car_dealers  
            -79.978469           1_sandwiches 1_restaurants 1_mediterranean  
            -79.

In [83]:
df.describe()

,lat_index,long_index,latitude,longitude,postal_code,review_count,stars,price,distance
count,4166.000000,4166.000000,4166.000000,4166.000000,4149.000000,4166.000000,4166.000000,4166.000000,4166.000000
mean,40.443063,-79.975987,40.443057,-79.975987,15215.661605,37.578012,3.701512,1.160586,3534.850282
std,0.019635,0.036074,0.019630,0.036074,11.330603,82.140316,0.927126,1.001508,2105.733330
min,40.384000,-80.077347,40.383611,-80.077347,15106.000000,3.000000,1.000000,0.000000,52.430472
25%,40.432000,-80.000732,40.431857,-80.000732,15206.000000,5.000000,3.000000,0.000000,1533.802317
50%,40.442000,-79.978481,40.442412,-79.978481,15216.000000,11.000000,4.000000,1.000000,3595.816471
75%,40.456000,-79.948210,40.455933,-79.948210,15222.000000,33.000000,4.500000,2.000000,5443.093110
max,40.508000,-79.913888,40.508240,-79.913888,15301.000000,1629.000000,5.000000,4.000000,6994.690490


In [21]:
blocks = pd.read_csv('PittsburghTrainRowCoords.csv',
          sep=';', compression='gzip', index_col='Unnamed: 0')
blocks.head()

,distance_from_city,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
420030103001007,1224.507208,U,40.434602,-79.993324,420030103001,9985.0,2465,2594,41768,0
420030103002002,818.745429,U,40.438604,-79.992094,420030103002,5028.0,3429,69,9209,0
420030103002015,959.100895,U,40.437214,-79.992321,420030103002,5028.0,3429,605,23201,0
420030103002016,1156.609093,U,40.435811,-79.990506,420030103002,5028.0,3429,2662,83841,0
420030103003003,1144.493890,U,40.438364,-79.985769,420030103003,12648.0,337,32,5624,0


In [22]:
blocks.describe()

,distance_from_city,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea
count,3297.000000,3297.000000,3297.000000,3.297000e+03,3270.000000,3297.000000,3297.000000,3.297000e+03,3297.000000
mean,3953.821516,40.443616,-79.990364,4.200325e+11,29455.965749,1066.621474,129.474977,2.700323e+04,4.363361
std,1391.131259,0.025048,0.036528,1.788758e+06,14155.041488,593.604076,347.559185,5.889944e+04,177.133087
min,181.704797,40.391750,-80.065701,4.200301e+11,5028.000000,152.000000,20.000000,1.075000e+03,0.000000
25%,2830.686965,40.423816,-80.017473,4.200310e+11,20428.000000,705.000000,31.000000,6.600000e+03,0.000000
50%,4077.792008,40.444409,-79.989016,4.200319e+11,27098.000000,944.000000,48.000000,1.068700e+04,0.000000
75%,5196.495408,40.463108,-79.959508,4.200330e+11,35115.000000,1292.000000,84.000000,2.285900e+04,0.000000
max,5999.687132,40.498095,-79.925021,4.200398e+11,125257.000000,4644.000000,3088.000000,1.225089e+06,7193.000000


In [23]:
make_X_train(blocks, df)

(2010, 2)
(2547, 2)
(2463, 2)
(1890, 2)
(1029, 2)
(259, 2)
(322, 2)
(351, 2)
(512, 2)
(365, 2)
(353, 2)
(381, 2)
(104, 2)
(174, 2)
(140, 2)
(126, 2)
(196, 2)
(240, 2)
(2595, 2)
(2622, 2)
(2694, 2)
(2681, 2)
(2673, 2)
(2604, 2)
(2579, 2)
(2582, 2)
(2673, 2)
(2683, 2)
(2701, 2)
(2655, 2)
(2702, 2)
(2654, 2)
(2571, 2)
(2734, 2)
(1928, 2)
(2621, 2)
(2505, 2)
(2753, 2)
(2750, 2)
(177, 2)
(287, 2)
(193, 2)
(261, 2)
(261, 2)
(261, 2)
(208, 2)
(368, 2)
(418, 2)
(390, 2)
(395, 2)
(384, 2)
(401, 2)
(493, 2)
(528, 2)
(565, 2)
(625, 2)
(1535, 2)
(1393, 2)
(1325, 2)
(1233, 2)
(339, 2)
(1269, 2)
(1325, 2)
(1201, 2)
(1095, 2)
(435, 2)
(1088, 2)
(181, 2)
(105, 2)
(168, 2)
(177, 2)
(221, 2)
(253, 2)
(383, 2)
(101, 2)
(118, 2)
(323, 2)
(1071, 2)
(188, 2)
(222, 2)
(217, 2)
(232, 2)
(220, 2)
(208, 2)
(209, 2)
(209, 2)
(191, 2)
(105, 2)
(123, 2)
(84, 2)
(102, 2)
(92, 2)
(100, 2)
(289, 2)
(258, 2)
(309, 2)
(376, 2)
(247, 2)
(285, 2)
(393, 2)
(387, 2)
(432, 2)
(380, 2)
(395, 2)
(365, 2)
(407, 2)
(454, 2)
(38

(54, 2)
(151, 2)
(55, 2)
(76, 2)
(204, 2)
(85, 2)
(87, 2)
(102, 2)
(63, 2)
(73, 2)
(75, 2)
(74, 2)
(71, 2)
(84, 2)
(85, 2)
(74, 2)
(84, 2)
(86, 2)
(86, 2)
(88, 2)
(87, 2)
(86, 2)
(86, 2)
(87, 2)
(86, 2)
(87, 2)
(86, 2)
(86, 2)
(86, 2)
(86, 2)
(103, 2)
(103, 2)
(93, 2)
(98, 2)
(97, 2)
(91, 2)
(182, 2)
(100, 2)
(124, 2)
(147, 2)
(130, 2)
(122, 2)
(106, 2)
(100, 2)
(100, 2)
(374, 2)
(344, 2)
(230, 2)
(226, 2)
(345, 2)
(338, 2)
(286, 2)
(214, 2)
(186, 2)
(103, 2)
(171, 2)
(173, 2)
(128, 2)
(103, 2)
(121, 2)
(114, 2)
(158, 2)
(163, 2)
(264, 2)
(153, 2)
(142, 2)
(42, 2)
(54, 2)
(93, 2)
(135, 2)
(117, 2)
(85, 2)
(63, 2)
(28, 2)
(34, 2)
(430, 2)
(373, 2)
(407, 2)
(380, 2)
(348, 2)
(236, 2)
(98, 2)
(164, 2)
(171, 2)
(142, 2)
(149, 2)
(61, 2)
(73, 2)
(33, 2)
(413, 2)
(473, 2)
(420, 2)
(243, 2)
(475, 2)
(472, 2)
(471, 2)
(418, 2)
(387, 2)
(90, 2)
(156, 2)
(272, 2)
(421, 2)
(451, 2)
(475, 2)
(476, 2)
(452, 2)
(423, 2)
(395, 2)
(339, 2)
(363, 2)
(398, 2)
(423, 2)
(453, 2)
(479, 2)
(480, 2)
(453, 2)

(48, 2)
(47, 2)
(34, 2)
(41, 2)
(36, 2)
(38, 2)
(38, 2)
(38, 2)
(40, 2)
(39, 2)
(42, 2)
(41, 2)
(40, 2)
(44, 2)
(53, 2)
(36, 2)
(37, 2)
(35, 2)
(35, 2)
(38, 2)
(38, 2)
(35, 2)
(36, 2)
(37, 2)
(35, 2)
(34, 2)
(34, 2)
(38, 2)
(38, 2)
(37, 2)
(37, 2)
(0, 2)
(0, 2)
(38, 2)
(39, 2)
(38, 2)
(38, 2)
(39, 2)
(0, 2)
(0, 2)
(41, 2)
(42, 2)
(21, 2)
(21, 2)
(21, 2)
(21, 2)
(19, 2)
(48, 2)
(46, 2)
(45, 2)
(47, 2)
(45, 2)
(28, 2)
(72, 2)
(37, 2)
(39, 2)
(44, 2)
(144, 2)
(126, 2)
(133, 2)
(71, 2)
(65, 2)
(73, 2)
(118, 2)
(67, 2)
(67, 2)
(75, 2)
(33, 2)
(42, 2)
(39, 2)
(33, 2)
(35, 2)
(32, 2)
(32, 2)
(36, 2)
(40, 2)
(38, 2)
(37, 2)
(20, 2)
(37, 2)
(38, 2)
(46, 2)
(41, 2)
(36, 2)
(43, 2)
(38, 2)
(39, 2)
(39, 2)
(39, 2)
(36, 2)
(35, 2)
(36, 2)
(43, 2)
(39, 2)
(36, 2)
(35, 2)
(42, 2)
(44, 2)
(44, 2)
(43, 2)
(45, 2)
(46, 2)
(44, 2)
(40, 2)
(39, 2)
(41, 2)
(41, 2)
(43, 2)
(40, 2)
(43, 2)
(43, 2)
(42, 2)
(42, 2)
(42, 2)
(38, 2)
(39, 2)
(42, 2)
(40, 2)
(45, 2)
(45, 2)
(45, 2)
(47, 2)
(46, 2)
(47, 2)
(50, 2)


(40, 2)
(39, 2)
(40, 2)
(53, 2)
(53, 2)
(41, 2)
(45, 2)
(41, 2)
(42, 2)
(45, 2)
(43, 2)
(42, 2)
(296, 2)
(213, 2)
(176, 2)
(177, 2)
(302, 2)
(321, 2)
(177, 2)
(301, 2)
(320, 2)
(319, 2)
(476, 2)
(411, 2)
(493, 2)
(470, 2)
(421, 2)
(489, 2)
(290, 2)
(25, 2)
(25, 2)
(26, 2)
(25, 2)
(70, 2)
(46, 2)
(52, 2)
(45, 2)
(37, 2)
(44, 2)
(49, 2)
(18, 2)
(21, 2)
(33, 2)
(20, 2)
(20, 2)
(23, 2)
(19, 2)
(23, 2)
(38, 2)
(23, 2)
(32, 2)
(29, 2)
(33, 2)
(28, 2)
(31, 2)
(184, 2)
(36, 2)
(221, 2)
(223, 2)
(219, 2)
(36, 2)
(222, 2)
(220, 2)
(67, 2)
(252, 2)
(249, 2)
(23, 2)
(31, 2)
(42, 2)
(46, 2)
(47, 2)
(181, 2)
(237, 2)
(225, 2)
(237, 2)
(237, 2)
(230, 2)
(223, 2)
(215, 2)
(25, 2)
(235, 2)
(234, 2)
(232, 2)
(219, 2)
(221, 2)
(223, 2)
(287, 2)
(253, 2)
(289, 2)
(289, 2)
(306, 2)
(306, 2)
(239, 2)
(242, 2)
(241, 2)
(288, 2)
(292, 2)
(35, 2)
(54, 2)
(56, 2)
(53, 2)
(2065, 2)
(348, 2)
(332, 2)
(355, 2)
(350, 2)
(389, 2)
(381, 2)
(676, 2)
(394, 2)
(402, 2)
(441, 2)
(516, 2)
(251, 2)
(276, 2)
(277, 2)
(278, 

In [24]:
# blocks.to_csv('CorpusPittsburghTrainRowCoords.csv',sep=';',compression='gzip')
blocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3297 entries, 420030103001007 to 15420039822001015
Data columns (total 11 columns):
distance_from_city    3297 non-null float64
urban                 3297 non-null object
latitude              3297 non-null float64
longitude             3297 non-null float64
blockgroup            3297 non-null int64
income                3270 non-null float64
blockgrouppop         3297 non-null int64
population            3297 non-null int64
landarea              3297 non-null int64
waterarea             3297 non-null int64
corpus                3297 non-null object
dtypes: float64(4), int64(5), object(2)
memory usage: 469.1+ KB


In [25]:
X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(blocks['corpus'])

In [26]:
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,8,83,3,78,3,36,0,10,0,0,2,23,0,33,1,50,0,17,0,7,3,131,3,139,1,43,0,1,0,0,0,5,0,3,0,27,0,11,0,1,6,289,0,...,0,0,0,0,0,1,45,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
1,0,0,72,27,42,70,12,57,7,6,0,0,6,54,25,10,30,71,2,18,0,18,120,36,114,71,23,79,0,2,0,0,0,4,0,2,9,17,0,14,1,0,247,97,0,...,0,0,0,0,0,35,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
2,0,0,60,35,36,69,10,48,3,10,0,0,4,50,25,12,30,64,2,18,0,16,119,34,109,72,20,77,0,1,0,0,0,4,0,3,7,17,0,14,1,0,238,98,0,...,0,0,0,0,0,34,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
3,0,0,7,75,8,69,6,30,1,9,0,0,1,23,0,30,2,48,0,13,0,7,5,125,9,125,1,44,0,2,0,0,0,6,0,3,0,27,0,12,1,0,18,257,0,...,0,0,0,0,0,1,45,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,1,20,9,28,5,8,1,6,0,0,1,4,0,25,2,30,0,5,0,0,3,81,9,106,1,15,0,0,0,0,2,2,1,0,0,4,1,1,1,0,24,124,0,...,0,0,0,0,0,1,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
joblib.dump(dfvect, 'X_trainPittsburghUpdated.joblib', compress=9)

['X_trainPittsburghUpdated.joblib']

In [94]:
y_trainPittsburgh = blocks['income']

In [95]:
joblib.dump(y_trainPittsburgh, 'y_trainPittsburgh.joblib', compress=9)

['y_trainPittsburgh.joblib']